In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Load your file with the proper encoding
df = pd.read_csv("superstore.csv", encoding="ISO-8859-1")

# Convert 'Order Date' and 'Ship Date' using dayfirst=True
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)

# Check for missing values
print(df.isnull().sum())

# Optional: Drop rows with missing values
df = df.dropna()

# View cleaned data
print(df.head())


# Create a new column for month-year
df['Month'] = df['Order Date'].dt.to_period('M')

# Group by month and sum sales
monthly_sales = df.groupby('Month')['Sales'].sum()

# Plot
monthly_sales.plot(kind='line', figsize=(12, 6), marker='o', title="Monthly Sales Trend")
plt.ylabel("Sales")
plt.xlabel("Month")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


# Category sales
category_sales = df.groupby('Category')['Sales'].sum().sort_values(ascending=False)

# Sub-category sales
subcat_sales = df.groupby('Sub-Category')['Sales'].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(12, 5))
sns.barplot(x=category_sales.index, y=category_sales.values)
plt.title("Sales by Category")
plt.ylabel("Sales")
plt.show()

plt.figure(figsize=(14, 6))
sns.barplot(x=subcat_sales.index, y=subcat_sales.values)
plt.title("Sales by Sub-Category")
plt.ylabel("Sales")
plt.xticks(rotation=45)
plt.show()
region_profit = df.groupby('Region')['Profit'].sum().sort_values()

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(x=region_profit.index, y=region_profit.values)
plt.title("Profit by Region")
plt.ylabel("Profit")
plt.grid(True)
plt.show()


import datetime as dt

# Use the latest order date as a reference
snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)

# Group by Customer ID
rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (snapshot_date - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
})

# Rename columns
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm.reset_index()

# View the result
rfm.head()

from sklearn.preprocessing import StandardScaler

# Extract the features
X = rfm[['Recency', 'Frequency', 'Monetary']]

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Try 1 to 10 clusters
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(8, 4))
plt.plot(range(1, 11), inertia, marker='o')
plt.title("Elbow Method - Optimal k")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

# Apply KMeans with 4 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(X_scaled)

# Define what each cluster means (customize these based on your results)
cluster_labels = {
    3: "High-Value Loyal Customers",
    0: "At-Risk Customers",
    1: "Inactive/Low-Value Customers",
    2: "New or Emerging Customers"
}
rfm['Segment'] = rfm['Cluster'].map(cluster_labels)

# Save the final data
rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean().round(1)

rfm.to_csv("clustered_rfmupdated.csv", index=False)
